# TP 01 - Fondamentaux NLP pour les Transformers

**Module** : Réseaux de Neurones Approfondissement  
**Durée** : 2h  
**Objectif** : Comprendre comment faire interpréter du texte par une machine

---

## Comment faire interpréter du texte par une machine ?

Vous avez déjà travaillé avec des **images** : une image est naturellement une grille de pixels, chaque pixel est un nombre (0-255). Le réseau peut directement les traiter.

Mais le **texte** ? 

```
"L'apprentissage automatique révolutionne l'intelligence artificielle"
```

Ce n'est qu'une suite de caractères. Un réseau de neurones ne comprend que des **nombres**. Comment passer de l'un à l'autre ?

---

## Les deux problèmes à résoudre

Pour transformer du texte en représentation numérique exploitable, il faut résoudre **deux problèmes distincts** :

### Problème 1 : La tokenization

**Comment découper le texte en morceaux ?**

```
"L'apprentissage automatique" → ???
```

Plusieurs stratégies possibles :
- Par mots : `["L'apprentissage", "automatique"]`
- Par caractères : `["L", "'", "a", "p", "p", "r", ...]`
- Par sous-mots : `["L'", "apprent", "issage", "automatique"]`

Chaque stratégie a ses avantages et inconvénients. Nous les explorerons dans ce TP.

### Problème 2 : L'embedding

**Comment transformer ces morceaux en vecteurs qui ont du SENS ?**

Une fois le texte découpé, on pourrait simplement numéroter les tokens :
```
"chat" → 42
"chien" → 73
"voiture" → 156
```

Mais ces nombres sont **arbitraires**. Ils ne capturent pas que "chat" et "chien" sont des concepts proches (animaux domestiques), alors que "voiture" est complètement différent.

**Il faut trouver un moyen** de transformer chaque token en un **vecteur de plusieurs dimensions** où la **proximité géométrique** reflète la **proximité sémantique** :

```
"chat"    → [0.2, -0.5, 0.8, ...]   ┐
                                    ├─ vecteurs proches !
"chien"   → [0.3, -0.4, 0.7, ...]   ┘

"voiture" → [-0.8, 0.2, -0.3, ...]  ← vecteur éloigné
```

Plusieurs approches existent pour construire ces vecteurs. Dans ce TP, nous explorerons **Word2Vec**, une méthode remarquable qui a révolutionné le NLP en 2013.

---

## Plan du TP

| Section | Problème traité | Ce que vous apprendrez |
|---------|-----------------|------------------------|
| §1-2 | Tokenization | Les 3 stratégies (mots, caractères, BPE) |
| §3-4 | Embedding | Comment les vecteurs capturent le sens (Word2Vec) |
| §5 | Bonus | Teaser sur le mécanisme d'attention |

Commençons par le premier problème : **comment découper le texte ?**

## 0. Installation et imports

Exécutez cette cellule pour installer les dépendances nécessaires.

In [ ]:
# Installation des dépendances (Google Colab)
!pip install torch matplotlib numpy gensim -q

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np

# Configuration
torch.manual_seed(42)
print(f"PyTorch version: {torch.__version__}")

---

# Partie 1 : La Tokenization

---

---

## 2. Tokenization : Découper le texte

La **tokenization** consiste à découper le texte en unités (tokens). Il existe plusieurs stratégies.

### 2.1 Tokenization par mots (Word-level)

La plus intuitive : on découpe sur les espaces et la ponctuation.

In [ ]:
# Tokenization simple par mots
def tokenize_words(text):
    """Tokenization basique par espaces et ponctuation."""
    import re
    # Sépare sur espaces et garde la ponctuation comme tokens
    tokens = re.findall(r"\w+|[^\w\s]", text.lower())
    return tokens

texte = "Le chat mange la souris. La souris court vite !"
tokens = tokenize_words(texte)

print(f"Texte : {texte}")
print(f"Tokens : {tokens}")
print(f"Nombre de tokens : {len(tokens)}")

**Problème** : Que faire avec un mot inconnu (hors vocabulaire) ?

```
Vocabulaire : ["le", "chat", "mange", "souris", ...]
Nouveau mot : "anticonstitutionnellement" → ???
```

### 2.2 Tokenization par caractères (Character-level)

Une solution : découper caractère par caractère. Plus de mots inconnus !

In [ ]:
def tokenize_chars(text):
    """Tokenization par caractères."""
    return list(text.lower())

texte = "Le chat dort."
tokens = tokenize_chars(texte)

print(f"Texte : {texte}")
print(f"Tokens : {tokens}")
print(f"Nombre de tokens : {len(tokens)}")

**Problème** : Séquences très longues ! "anticonstitutionnellement" = 25 tokens.

Le modèle doit "réapprendre" que `c-h-a-t` forme le concept de chat.

### 2.3 Tokenization Subword : BPE (Byte Pair Encoding)

Les deux approches précédentes ont des défauts :
- **Mots** : vocabulaire énorme + mots inconnus
- **Caractères** : séquences trop longues + perte de sens

**BPE** (Byte Pair Encoding) est un **compromis intelligent** utilisé par GPT, BERT, et tous les LLMs modernes.

---

#### Le principe

BPE construit son vocabulaire en analysant un grand corpus de texte :

1. **Départ** : vocabulaire = tous les caractères
2. **Répéter** : trouver la paire de tokens adjacents la plus fréquente → la fusionner en un nouveau token
3. **Stop** : quand le vocabulaire atteint la taille voulue (ex: 50 000 tokens)

---

#### L'algorithme pas à pas

Pour comprendre, prenons un **corpus artificiel simplifié** :
```
"smartphone smartphone smartphone smartwatch smartwatch phone phone"
```

**Étape 0 : Partir des caractères**
```
Vocabulaire : {s, m, a, r, t, p, h, o, n, e, w, c}
```

**Étapes suivantes : Fusionner les paires les plus fréquentes**

| Étape | Paire la + fréquente | Nouveau token |
|-------|---------------------|---------------|
| 1 | (s, m) | "sm" |
| 2 | (sm, a) | "sma" |
| 3 | (sma, r) | "smar" |
| 4 | (smar, t) | "smart" |
| 5 | (p, h) | "ph" |
| 6 | (ph, o) | "pho" |
| 7 | (pho, n) | "phon" |
| 8 | (phon, e) | "phone" |

**Résultat :**
```
"smartphone" → ["smart", "phone"]  ← 2 tokens réutilisables !
"smartwatch" → ["smart", "watch"]
"phone"      → ["phone"]
```

---

#### Pourquoi c'est malin ?

Un mot **jamais vu** comme "smartcar" sera découpé en :
```
"smartcar" → ["smart", "car"]
```

Le modèle connaît déjà "smart" ! Pas besoin de token `<UNK>`.

**Le meilleur des deux mondes** :
- Mots fréquents → tokens entiers (efficace)
- Mots rares/nouveaux → sous-mots connus (robuste)

**Ressource** : [Explication détaillée des tokenizers (FR)](https://lbourdois.github.io/blog/nlp/Les-tokenizers/)

In [ ]:
# Démonstration avec un vrai tokenizer (GPT-2)
!pip install transformers -q

from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

textes = [
    "Le chat mange.",
    "anticonstitutionnellement",
    "Hello world!",
    "Transformers are amazing!"
]

print("=== Tokenization BPE (GPT-2) ===")
for texte in textes:
    tokens = tokenizer.tokenize(texte)
    ids = tokenizer.encode(texte)
    print(f"\n'{texte}'")
    print(f"  Tokens : {tokens}")
    print(f"  IDs    : {ids}")

**Observations** :
- Les mots courants anglais sont souvent des tokens uniques
- Les mots français/rares sont découpés
- Le caractère `Ġ` indique un espace avant le token

### Exercice 1 : Comparer les tokenizations

In [ ]:
# ============================================
# EXERCICE 1 : Comparer les tokenizations
# ============================================

texte_test = "L'intelligence artificielle révolutionne le monde."

# TODO: Tokenizer avec les 3 méthodes et comparer le nombre de tokens

# 1. Par mots
tokens_mots = None  # tokenize_words(texte_test)

# 2. Par caractères  
tokens_chars = None  # tokenize_chars(texte_test)

# 3. BPE (GPT-2)
tokens_bpe = None  # tokenizer.tokenize(texte_test)

print(f"Texte : {texte_test}")
print(f"\nMots      : {len(tokens_mots) if tokens_mots else '?'} tokens")
print(f"Caractères: {len(tokens_chars) if tokens_chars else '?'} tokens")
print(f"BPE       : {len(tokens_bpe) if tokens_bpe else '?'} tokens")

### 2.4 Construction d'un vocabulaire

Une fois la stratégie choisie, on construit un **vocabulaire** : une table de correspondance token ↔ index.

In [ ]:
# Construire un vocabulaire simple
corpus = [
    "le chat mange",
    "le chien dort",
    "la souris court"
]

# Collecter tous les tokens uniques
all_tokens = set()
for phrase in corpus:
    all_tokens.update(tokenize_words(phrase))

# Créer le vocabulaire avec tokens spéciaux
vocab = {"<PAD>": 0, "<UNK>": 1}  # Padding et Unknown
for i, token in enumerate(sorted(all_tokens)):
    vocab[token] = i + 2

# Vocabulaire inverse
id_to_token = {v: k for k, v in vocab.items()}

print("Vocabulaire :")
for token, idx in vocab.items():
    print(f"  {idx}: '{token}'")

# Encoder une phrase
def encode(text, vocab):
    return [vocab.get(t, vocab["<UNK>"]) for t in tokenize_words(text)]

def decode(ids, id_to_token):
    return [id_to_token[i] for i in ids]

phrase = "le chat court"
encoded = encode(phrase, vocab)
decoded = decode(encoded, id_to_token)

print(f"\nPhrase : '{phrase}'")
print(f"Encodé : {encoded}")
print(f"Décodé : {decoded}")

---

## 3. Embeddings : Des indices aux vecteurs

### Le problème des indices

Les indices (0, 1, 2, ...) n'ont pas de **sens sémantique**. 

- `chat = 3` et `chien = 4` → sont-ils proches ? (oui, ce sont des animaux)
- `chat = 3` et `voiture = 42` → sont-ils proches ? (non)

Mais avec des indices, on ne peut pas mesurer cette proximité !

### La solution : Embeddings

On associe à chaque token un **vecteur dense** de dimension fixe (ex: 128, 256, 768).

```
"chat"    → [0.2, -0.5, 0.8, 0.1, ...] (128 dimensions)
"chien"   → [0.3, -0.4, 0.7, 0.2, ...] (proche de chat !)
"voiture" → [-0.8, 0.2, -0.3, 0.9, ...] (loin de chat)
```

Ces vecteurs sont **appris** pendant l'entraînement du modèle.

In [ ]:
# Démonstration : Embedding en PyTorch
vocab_size = 10  # 10 tokens dans notre vocabulaire
embed_dim = 4    # Chaque token → vecteur de dimension 4

# Créer une couche d'embedding
embedding = nn.Embedding(vocab_size, embed_dim)

print(f"Matrice d'embedding : {embedding.weight.shape}")
print(f"  → {vocab_size} tokens × {embed_dim} dimensions\n")

# Récupérer l'embedding d'un token
token_id = torch.tensor([3])  # Token d'indice 3
vector = embedding(token_id)

print(f"Token 3 → {vector}")

In [ ]:
# Embedding d'une séquence complète
sequence = torch.tensor([2, 5, 3, 7])  # 4 tokens
embedded = embedding(sequence)

print(f"Séquence : {sequence.tolist()}")
print(f"Shape après embedding : {embedded.shape}")  # (4, 4)
print(f"\nVecteurs :\n{embedded}")

### Mesurer la similarité : Distance cosinus

Avec des vecteurs, on peut mesurer la **similarité** entre mots :

$$\text{similarité}(A, B) = \cos(\theta) = \frac{A \cdot B}{\|A\| \|B\|}$$

- Résultat entre -1 (opposés) et 1 (identiques)
- 0 = orthogonaux (pas de relation)

In [ ]:
def cosine_similarity(a, b):
    """Calcule la similarité cosinus entre deux vecteurs."""
    return torch.dot(a, b) / (torch.norm(a) * torch.norm(b))

# Exemple avec nos embeddings aléatoires
vec_2 = embedding(torch.tensor(2))
vec_3 = embedding(torch.tensor(3))
vec_7 = embedding(torch.tensor(7))

sim_2_3 = cosine_similarity(vec_2, vec_3)
sim_2_7 = cosine_similarity(vec_2, vec_7)

print(f"Similarité(token 2, token 3) = {sim_2_3:.4f}")
print(f"Similarité(token 2, token 7) = {sim_2_7:.4f}")
print("\n(Valeurs aléatoires car embeddings non entraînés)")

---

## 4. Word2Vec : Apprendre des embeddings qui ont du sens

Jusqu'ici, nos embeddings étaient **aléatoires** (initialisés au hasard dans PyTorch). Comment obtenir des vecteurs où "chat" et "chien" sont vraiment proches ?

**Word2Vec** (Mikolov et al., 2013) a révolutionné le NLP en montrant qu'on peut **apprendre** des embeddings à partir de texte brut, sans supervision.

---

### L'intuition fondamentale

> **"Tu connais un mot par les mots qui l'entourent"** (hypothèse distributionnelle)

Observez ces phrases :
```
"Le chat mange sa pâtée"
"Le chat dort sur le canapé"  
"Mon chat joue avec une balle"

"Le chien mange sa pâtée"
"Le chien dort sur le canapé"
"Mon chien joue avec une balle"
```

"Chat" et "chien" apparaissent dans les **mêmes contextes**. Word2Vec va leur attribuer des vecteurs similaires.

---

### Comment Word2Vec apprend ? (Skip-gram)

L'idée est simple : entraîner un réseau à **prédire les mots du contexte** à partir d'un mot central.

```
Phrase : "Le chat noir dort paisiblement"
                  ↑
            mot central

Fenêtre de contexte (±2 mots) :
    Entrée : "noir"
    Cibles : ["chat", "dort"] (le réseau essaie de les prédire)
```

**Architecture simplifiée :**

```
    "noir"                      "chat" ?
      ↓                            ↑
 ┌──────────┐                ┌──────────┐
 │ Embedding │  → vecteur →  │ Prédiction│
 │ (lookup)  │    256 dim    │ (softmax) │
 └──────────┘                └──────────┘
```

**Pendant l'entraînement :**
- Le réseau voit des millions de paires (mot central, mot contexte)
- Il ajuste les embeddings pour que les mots apparaissant dans les mêmes contextes aient des vecteurs proches
- Les embeddings **émergent** de cette tâche de prédiction

---

### Pourquoi les analogies marchent ?

Après entraînement, les vecteurs encodent des **relations** :

```
vecteur("roi") - vecteur("homme") ≈ vecteur("reine") - vecteur("femme")
```

Autrement dit, la "direction" homme→femme dans l'espace vectoriel est la même que roi→reine :

```
        homme ─────────────→ femme
          ↑    (même         ↑
          │   direction)     │
         roi ─────────────→ reine
```

C'est pour ça que `roi - homme + femme ≈ reine` fonctionne !

---

### Chargeons un modèle pré-entraîné

Nous allons utiliser **GloVe** (similaire à Word2Vec), entraîné sur Wikipedia.

In [ ]:
# Charger un modèle Word2Vec pré-entraîné
import gensim.downloader as api

print("Chargement du modèle Word2Vec (peut prendre 1-2 min)...")
model = api.load("glove-wiki-gigaword-100")  # 100 dimensions, entraîné sur Wikipedia
print(f"Modèle chargé ! Vocabulaire : {len(model)} mots")

In [ ]:
# Explorer les similarités
print("=== Mots similaires à 'king' ===")
for word, score in model.most_similar("king", topn=5):
    print(f"  {word}: {score:.4f}")

print("\n=== Mots similaires à 'computer' ===")
for word, score in model.most_similar("computer", topn=5):
    print(f"  {word}: {score:.4f}")

In [ ]:
# ============================================
# EXERCICE 2 : Explorer les similarités
# ============================================

# TODO: Trouver les 5 mots les plus similaires à :
# - "france"
# - "cat" (chat en anglais)
# - "happy"

# Exemple :
# for word, score in model.most_similar("france", topn=5):
#     print(f"  {word}: {score:.4f}")

In [ ]:
# La magie des analogies : king - man + woman = ?
print("=== Analogie : king - man + woman = ? ===")
result = model.most_similar(positive=["king", "woman"], negative=["man"], topn=3)
for word, score in result:
    print(f"  {word}: {score:.4f}")

print("\n=== Analogie : paris - france + italy = ? ===")
result = model.most_similar(positive=["paris", "italy"], negative=["france"], topn=3)
for word, score in result:
    print(f"  {word}: {score:.4f}")

In [ ]:
# ============================================
# EXERCICE 3 : Trouver des analogies
# ============================================

# TODO: Tester ces analogies (et en inventer d'autres !)
# - "berlin" - "germany" + "france" = ?
# - "good" - "better" + "bad" = ?
# - "cat" - "kitten" + "dog" = ?

# Syntaxe :
# model.most_similar(positive=["A", "C"], negative=["B"], topn=3)
# Pour calculer A - B + C

In [ ]:
# Visualisation des embeddings en 2D
!pip install scikit-learn -q
from sklearn.decomposition import PCA

# Sélectionner quelques mots
words = ["king", "queen", "man", "woman", "prince", "princess",
         "cat", "dog", "lion", "tiger",
         "car", "bus", "train", "plane"]

# Récupérer leurs vecteurs
vectors = np.array([model[w] for w in words])

# Réduire à 2D avec PCA
pca = PCA(n_components=2)
vectors_2d = pca.fit_transform(vectors)

# Visualiser
plt.figure(figsize=(12, 8))
plt.scatter(vectors_2d[:, 0], vectors_2d[:, 1], c='blue', s=100)

for i, word in enumerate(words):
    plt.annotate(word, (vectors_2d[i, 0] + 0.1, vectors_2d[i, 1] + 0.1), fontsize=12)

plt.title("Embeddings Word2Vec projetés en 2D")
plt.xlabel("Composante 1")
plt.ylabel("Composante 2")
plt.grid(True, alpha=0.3)
plt.show()

print("Observation : Les mots de même catégorie sont regroupés !")

---

## 5. Teaser : Le mécanisme d'attention

Maintenant que nous savons représenter du texte (tokenization + embeddings), la prochaine étape est de permettre aux mots de **communiquer entre eux**.

C'est le rôle du **mécanisme d'attention**, que nous verrons en détail au prochain TP.

### L'idée

> Pour comprendre un mot, il faut regarder les autres mots de la phrase.

Exemple : *"Le chat qui dormait sur le canapé a sauté"*
- Pour comprendre **"a sauté"** → regarder **"chat"** (le sujet)
- Pour comprendre **"dormait"** → regarder **"chat"** et **"canapé"**

### Visualisation

In [ ]:
# Matrice d'attention simulée
phrase = ["Le", "chat", "mange", "la", "souris"]

# Chaque ligne = un mot qui "regarde" les autres
# Valeurs = poids d'attention (somme = 1 par ligne)
attention = torch.tensor([
    [0.8, 0.1, 0.05, 0.03, 0.02],  # "Le" regarde surtout lui-même
    [0.1, 0.7, 0.1, 0.05, 0.05],   # "chat" regarde surtout lui-même
    [0.05, 0.4, 0.4, 0.05, 0.1],   # "mange" regarde "chat" et lui-même
    [0.02, 0.03, 0.05, 0.8, 0.1],  # "la" regarde surtout lui-même
    [0.02, 0.1, 0.2, 0.08, 0.6],   # "souris" regarde "mange" et elle-même
])

# Visualisation
plt.figure(figsize=(8, 6))
plt.imshow(attention, cmap='Blues')
plt.xticks(range(5), phrase)
plt.yticks(range(5), phrase)
plt.xlabel("Mots regardés")
plt.ylabel("Mots qui regardent")
plt.title("Qui regarde qui ? (Matrice d'attention)")
plt.colorbar(label="Poids d'attention")

for i in range(5):
    for j in range(5):
        plt.text(j, i, f'{attention[i,j]:.2f}', 
                ha='center', va='center',
                color='white' if attention[i,j] > 0.5 else 'black')
plt.show()

print("Le verbe 'mange' regarde fortement 'chat' (son sujet) !")

**Ce qu'on voit** :
- Chaque mot peut "regarder" tous les autres mots
- Les poids indiquent l'importance de chaque relation
- Le modèle **apprend** ces poids pendant l'entraînement

**Au prochain TP**, nous verrons :
- Pourquoi les RNN/LSTM ont des limites
- Comment calculer cette matrice d'attention
- Les concepts Query, Key, Value

---

## 6. Récapitulatif

### Le pipeline NLP

| Étape | Entrée | Sortie | Rôle |
|-------|--------|--------|------|
| **Tokenization** | Texte brut | Liste de tokens | Découper le texte |
| **Vocabulaire** | Tokens | Indices | Table token ↔ ID |
| **Embedding** | Indices | Vecteurs denses | Sens sémantique |

### Points clés

1. **Tokenization BPE** : meilleur compromis entre mots et caractères
2. **Embeddings** : transforment les mots en vecteurs comparables
3. **Word2Vec** : montre que les embeddings capturent le sens (analogies !)
4. **Similarité cosinus** : mesure la proximité entre vecteurs

### Prochaine session

Nous verrons le **mécanisme d'attention** : comment les mots "communiquent" entre eux pour se comprendre mutuellement.

---

## 7. Pour aller plus loin (optionnel)

### Ressources

- [Les tokenizers en NLP (FR)](https://lbourdois.github.io/blog/nlp/Les-tokenizers/) - Excellent article en français
- [The Illustrated Word2Vec](https://jalammar.github.io/illustrated-word2vec/) - Visualisations très claires

### Expérimentations suggérées

1. Tester d'autres analogies Word2Vec
2. Visualiser les embeddings de votre choix
3. Comparer différents tokenizers (BERT, GPT-2, etc.)